In [1]:
import sys
sys.path.append("..")

In [2]:
from collections import defaultdict, namedtuple
import re

from resources.utils import get_puzzle_input

### Part 1

Each Elf has made a claim about which area of fabric would be ideal for Santa's suit. All claims have an ID and consist of a single rectangle with edges parallel to the edges of the fabric. Each claim's rectangle is defined as follows:

The number of inches between the left edge of the fabric and the left edge of the rectangle.

The number of inches between the top edge of the fabric and the top edge of the rectangle.

The width of the rectangle in inches.

The height of the rectangle in inches.

A claim like #123 @ 3,2: 5x4 means that claim ID 123 specifies a rectangle 3 inches from the left edge, 2 inches from the top edge, 5 inches wide, and 4 inches tall. Visually, it claims the square inches of fabric represented by # (and ignores the square inches of fabric represented by .) in the diagram below:

```
...........
...........
...#####...
...#####...
...#####...
...#####...
...........
...........
...........
```

The problem is that many of the claims overlap, causing two or more claims to cover part of the same areas. For example, consider the following claims:

```
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2
Visually, these claim the following areas:

........
...2222.
...2222.
.11XX22.
.11XX22.
.111133.
.111133.
........
```

The four square inches marked with X are claimed by both 1 and 2. (Claim 3, while adjacent to the others, does not overlap either of them.)

If the Elves all proceed with their own plans, none of them will have enough fabric. How many square inches of fabric are within two or more claims?

In [3]:
Claim = namedtuple('Claim', ['id', 'x', 'y', 'width', 'height'])

claim_re = re.compile(r'^#([0-9]+) @ ([0-9]+),([0-9]+): ([0-9]+)x([0-9]+)')

def parse_claim(str):
    match = claim_re.match(str)
    if not match:
        raise ValueError('Invalid claim {}'.format(str))
    
    return Claim(*(int(x) for x in match.groups()))    

In [4]:
assert claim_re.match('#113 @ 1,3: 4x4').groups() == ('113', '1', '3', '4', '4')
assert parse_claim('#113 @ 1,3: 4x5') == Claim(id=113, x=1, y=3, width=4, height=5)

In [5]:
def squares_in_claim(claim):
    for x in range(claim.x, claim.x + claim.width):
        for y in range(claim.y, claim.y + claim.height):
            yield(x, y)

In [ ]:
claim = Claim(id=113, x=1, y=3, width=2, height=3)
assert set(squares_in_claim(claim)) == {(1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5)}

In [6]:
def common_squares(claims):
    squares = defaultdict(lambda *_: 0)
    
    for claim in claims:
        for square in squares_in_claim(claim):
            squares[square] += 1
    
    return [square for square, used in squares.items() if used > 1]

In [7]:
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2

claims = [
    Claim(id=1, x=1, y=3, width=4, height=4),
    Claim(id=2, x=3, y=1, width=4, height=4),
    Claim(id=3, x=5, y=5, width=2, height=2)
]

assert set(common_squares(claims)) == {(3, 3), (3, 4), (4, 3), (4, 4)}

In [8]:
puzzle_input = get_puzzle_input('/tmp/aoc_day_3.txt')
claims = [parse_claim(str) for str in puzzle_input]

In [9]:
len(common_squares(claims))

119572

### Part 2

Amidst the chaos, you notice that exactly one claim doesn't overlap by even a single square inch of fabric with any other claim. If you can somehow draw attention to it, maybe the Elves will be able to make Santa's suit after all!

For example, in the claims above, only claim 3 is intact after all claims are made.

What is the ID of the only claim that doesn't overlap?

In [10]:
def non_overlapping_claim(claims):
    all_claims = {claim.id for claim in claims}
    overlapping_claims = set()

    squares = defaultdict(list)
    
    for claim in claims:
        for square in squares_in_claim(claim):
            squares[square].append(claim.id)
            if len(squares[square]) > 1:
                overlapping_claims |= set(squares[square])
    
    return all_claims - overlapping_claims   

In [11]:
non_overlapping_claim(claims)

{775}